**Crawling data** merupakan sebuah teknik pengumpulan data dimana data tidak ditulis secara manual melainkan menggunakan mesin. Salah satu library yang dapat digunakan untuk melakukan crawling adalah dengan menggunakan twint. twint merupakan libraby yang dapat digunakan untuk melakukan crawling data pada twitter. adapun persiapan yang harus dilakukan terlebih dahulu adalah menginstall jupyter-book sebagai tools untuk membuat konten.

Pertama-tama masuk ke folder webmining terlebih dahulu

Kemudian install twin dari github dengan code seperti berikut:

In [ ]:
!pip install git+https://github.com/twintproject/twint.git

Setelah itu install juga nest asynco

In [ ]:
!pip install nest_asyncio

In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply() #digunakan sekali untuk mengaktifkan tindakan serentak dalam notebook jupyter.

Dalam melakukan crawl menggunakan **twint** ada beberapa baris code yang dituliskan. Dibawah ini merupakan code untuk melakukan crawl menggunakan twint:

In [ ]:
import twint
c = twint.Config()

c.Since = '2022-10-11'
c.Until = '2022-10-12'
c.Search = 'tragedi kanjuruhan'
c.Output = 'data_tweet.csv'

twint.run.Search(c)

Setelah data dicrawling selanjutnya dilakukan pra preprocessing atau bisa juga disebut **data cleaning**. Apa itu **data cleaning** ? data sudah bersih yang dimana pada data tersebut tidak terdapat url, emoji, backslice dan yang lainnya. Tujuan dari data cleaning yaitu agar data yang masuk kedalam mesin itu murni berupa data teks sehingga mesin lebih mudah dalam melakukan pembelajaran.

In [ ]:
import pandas as pd
import numpy as np
data_tweet = pd.read_csv('dataospektweet.csv') #membaca file csv dengan nama output
data_tweet.head()

Selanjutnya adalah mengganti data index 1 secara manual untuk menghapus emoji karena emoji tersebut tidak valid sehingga tidak bisa dihapus menggunakan mesin

In [ ]:
data_tweet["tweet"][1] = "Cuz i don't have a proper weekend...   Terus nyadar kalau minggu depan jg ada ospek offline"
data_tweet["tweet"][1]

**Case Folding**

In [ ]:
data_tweet["tweet"] = data_tweet["tweet"].str.lower()
data_tweet["tweet"]

**Cleaning Data**

In [ ]:
import nltk
import string
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

Yang pertama adalah menghapus url

In [ ]:
def remove_url(text):
  return re.sub(r'http\S+', '', text)
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_url)

Selanjutnya adalah menghapus karekter spesial

In [ ]:
def remove_tweet_special(text):
  #remove tab, new line, and back slice
  text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
  #remove non ASCII (emoticon,chinese word,.etc)
  text = text.encode('ascii','replace').decode('ascii')
  #remove mention, link, hastag
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\s+)", " ", text).split())
  #remove incomplete url
  return text.replace("http://"," ").replace("https://"," ")
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_tweet_special)

In [ ]:
def remove_number(text):
  return re.sub(r"\d+", "",text)
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_number)

In [ ]:
def remove_punctuation(text):
  return text.translate(str.maketrans("","",string.punctuation))
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_punctuation)

In [ ]:
def remove_whitespace(text):
  return text.strip()
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_whitespace)

In [ ]:
def remove_singl_char(text):
  return re.sub(r"\b[a-zA-Z]\b","",text)
data_tweet["tweet"] = data_tweet["tweet"].apply(remove_singl_char)

In [ ]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)
data_tweet["tweet_tokens"] = data_tweet["tweet"].apply(word_tokenize_wrapper)

In [ ]:
data_tweet["tweet"].head()

In [ ]:
data_tweet["tweet_tokens"].head()

In [ ]:
def freqDist_wrapper(text):
  return FreqDist(text)

data_tweet["tweet_tokens_fdist"] = data_tweet["tweet_tokens"].apply(freqDist_wrapper)
data_tweet["tweet_tokens_fdist"].head()

**Stopwrord Removal**

In [ ]:
#import nltk untuk melakukan peprocessing data
import nltk
nltk.download('stopwords') #download stopword indonesia
from nltk.corpus import stopwords
stopword_language = stopwords.words('indonesian') #set stopword
stopword_language.extend(['jg','jos','yg','dg','dgn','rt','ny','d','klo','kalo',
                          'amp','biar','bikin','bilang','gak','ga','krn','nya','nih',
                          'sih','si','tau','tdk','tuh','utk','ya','jd','jgn','sdh','aja',
                          'n','t',
                          ])
txt_stopword = pd.read_csv('dataospektweet.csv',names=['stopwords'],header=None)
stopword_language.extend(txt_stopword['stopwords'][0].split(' '))

In [ ]:
def stopword_removal(words):
  return [word for word in words if word not in stopword_language]

data_tweet["tweet_tokens_rstopword"] = data_tweet["tweet_tokens"].apply(stopword_removal)

**Normalisasi**

In [ ]:
normalized_word = pd.read_csv('data_tweet_new.csv')
normalized_word_dict = {}

for index,row in normalized_word_dict:
  if row[0] not in normalized_word_dict:
    normalized_word_dict[row[0]]=row[1]

def normalized_term(document):
  return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

data_tweet["tweet_normalized"] = data_tweet["tweet_tokens_rstopword"].apply(normalized_term)
data_tweet["tweet_normalized"].head()

**Stemming**

In [ ]:
!pip install sastrawi

In [ ]:
!pip install swifter

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in data_tweet['tweet_normalized']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)

print(term_dict)
print("------------------------")

In [ ]:
def get_stemmed_term(document):
  return [term_dict[term] for term in document]

data_tweet["tweet_tokens_stemmed"] = data_tweet["tweet_normalized"].swifter.apply(get_stemmed_term)

In [ ]:
data_tweet.to_csv("preprocessing_tweet.csv")

**Membuat VSM**

In [ ]:
!pip install sklearn

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
titles = pd.read_csv('preprocessing_tweet.csv',sep=',',usecols=["tweet_tokens_stemmed"],squeeze=True)

docs = titles.values
bag = count.fit_transform(docs)
print(docs)

["['kocak', 'banget', 'gue', 'ospek']"
 "['cuz', 'dont', 'have', 'proper', 'weekend', 'nyadar', 'minggu', 'ospek', 'offline']"
 "['ootd', 'ospek', 'gueee', 'gajelasss', 'bangettt', 'weeee', 'astagaaaa']"
 "['ospek', 'with', 'cupy']" "['gladi', 'bersih', 'ospek']"
 "['lu', 'gimanaaa']"
 "['capek', 'banget', 'habis', 'ospek', 'jurusanmakrab']"
 "['ospek', 'ngga', 'kenal']"
 "['maksud', 'surat', 'kantor', 'hrd', 'ospek', 'izin', 'gitu']"
 "['ospek', 'yu', 'yu']"
 "['ospek', 'onlen', 'mah', 'diem', 'aje', 'ye', 'wak', 'ye']"
 "['gue', 'hrs', 'ospek', 'susul', 'anjg']"
 "['jalan', 'pintas', 'jembatan', 'btw', 'uty', 'kayak', 'ospek', 'pake', 'item', 'putih']"
 "['adbis', 'ospek']" "['semangat', 'ospek']"
 "['abis', 'ospek', 'lgsg', 'kelas', 'jam', 'tuju', 'anjir', 'strong', 'girls', 'stream', 'born', 'pink']"
 "['semangat', 'ospek']"
 "['souieii', 'oalah', 'kira', 'kotak', 'bekel', 'is', 'must', 'ospek', 'lo', 'cia']"
 "['angkat', 'ospek', 'onlen', 'tp', 'pas', 'ngospek', 'sok', 'rad']"
 "[

In [ ]:
print(count.vocabulary_)

{'kocak': 342, 'banget': 56, 'gue': 222, 'ospek': 480, 'cuz': 124, 'dont': 158, 'have': 243, 'proper': 517, 'weekend': 668, 'nyadar': 461, 'minggu': 407, 'offline': 467, 'ootd': 475, 'gueee': 223, 'gajelasss': 185, 'bangettt': 57, 'weeee': 667, 'astagaaaa': 43, 'with': 670, 'cupy': 119, 'gladi': 214, 'bersih': 74, 'lu': 371, 'gimanaaa': 208, 'capek': 107, 'habis': 231, 'jurusanmakrab': 296, 'ngga': 448, 'kenal': 325, 'maksud': 382, 'surat': 598, 'kantor': 310, 'hrd': 259, 'izin': 276, 'gitu': 211, 'yu': 681, 'onlen': 473, 'mah': 378, 'diem': 145, 'aje': 15, 'ye': 676, 'wak': 664, 'hrs': 260, 'susul': 600, 'anjg': 28, 'jalan': 280, 'pintas': 506, 'jembatan': 288, 'btw': 99, 'uty': 659, 'kayak': 315, 'pake': 489, 'item': 272, 'putih': 522, 'adbis': 6, 'semangat': 558, 'abis': 4, 'lgsg': 359, 'kelas': 320, 'jam': 281, 'tuju': 638, 'anjir': 31, 'strong': 595, 'girls': 210, 'stream': 593, 'born': 91, 'pink': 505, 'souieii': 589, 'oalah': 465, 'kira': 335, 'kotak': 346, 'bekel': 67, 'is': 27

In [ ]:
print(count.get_feature_names())
a = count.get_feature_names()

['aaa', 'aaaa', 'aaaaaaaa', 'abang', 'abis', 'acara', 'adbis', 'adek', 'adik', 'aduh', 'aga', 'agustus', 'airen', 'aj', 'ajar', 'aje', 'ajg', 'ak', 'aku', 'akun', 'albedo', 'alis', 'allah', 'anak', 'and', 'aneh', 'angkat', 'anjay', 'anjg', 'anjgggg', 'anjingg', 'anjir', 'anjirr', 'anjng', 'anjoyy', 'anjrit', 'anw', 'apo', 'april', 'apus', 'around', 'as', 'asa', 'astagaaaa', 'asu', 'at', 'atur', 'au', 'awakdewe', 'awake', 'baca', 'baddies', 'baja', 'baju', 'balon', 'bang', 'banget', 'bangettt', 'bangun', 'banya', 'banyak', 'bareng', 'bawa', 'bc', 'be', 'been', 'begadang', 'bekel', 'bekelny', 'belah', 'beli', 'benci', 'berangkat', 'berita', 'bersih', 'beso', 'besok', 'besokk', 'bestie', 'bg', 'bgst', 'bgt', 'bilang', 'bilek', 'blm', 'block', 'bngt', 'bojoku', 'bola', 'bom', 'boom', 'born', 'bos', 'bosen', 'bowlny', 'br', 'bru', 'bs', 'bsk', 'btw', 'bubuub', 'buka', 'bunda', 'bungkus', 'calon', 'campur', 'cantik', 'capek', 'capk', 'cenenk', 'cepat', 'cerdas', 'cia', 'cie', 'ciee', 'cipta'

In [ ]:
print(bag.toarray())
b = bag.toarray()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
dfa = pd.DataFrame(data=a)
num_rows = -1

for row in open("preprocessing_tweet.csv"):
  num_rows+=1

dfb = pd.DataFrame(data=b,index=range(0,num_rows),columns=[a])
dfb

,aaa,aaaa,aaaaaaaa,abang,abis,acara,adbis,adek,adik,aduh,...,yaa,ye,ygy,yh,yhh,youve,yu,yuhuww,yuk,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**K-means Clustering** Merupakan sebuah metode untuk mengelompokkan data berdasarkan kemiripan dari data tersebut. berbeda dengan klasifikasi clustering mengelompokkan data yang unsupervised(tidak terawasi) atau belum memiliki label.

In [ ]:
from sklearn.cluster import KMeans

kmeans =KMeans(n_clusters=2)
kmeans=kmeans.fit(dfb)
prediksi=kmeans.predict(dfb)
centroids = kmeans.cluster_centers_
prediksi
data=pd.DataFrame(prediksi,columns=["Cluster"])
data

In [ ]:
dflabels = pd.read_csv('label.csv')
dfb['labels'] = dflabels
dfb

,aaa,aaaa,aaaaaaaa,abang,abis,acara,adbis,adek,adik,aduh,...,ye,ygy,yh,yhh,youve,yu,yuhuww,yuk,zoom,labels
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positif
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negatif
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negatif
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,netral
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,netral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negatif
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negatif
186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,netral
187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positif


**Membagi data training dan data testing**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(dfb.drop(labels=['labels'], axis=1),
    dfb['labels'],
    test_size=0.3,
    random_state=0)

In [ ]:
y_test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
testing=[]
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train.values, y_train)
Y_pred = neigh.predict(X_test.values) 
Y_pred

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score,precision_score
neigh.predict(X_test.values) 
accuracy_neigh=round(accuracy_score(y_test,Y_pred)* 100, 2)
acc_neigh = round(neigh.score(X_train.values, y_train) * 100, 2)
acc_neigh

**Sort**

In [ ]:
#import pembagian train dan test dari sklearn model selection
from sklearn.model_selection import train_test_split
#import mutual information untuk klasifikasi
from sklearn.feature_selection import mutual_info_classif
#set data training dan data testing dari data
X_train,X_test,y_train,y_test=train_test_split(dfb.drop(labels=['labels'], axis=1),
    dfb['labels'],
    test_size=0.3,
    random_state=0)
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

# Text Summarization

In [ ]:
import pandas as pd
data_berita = pd.read_csv('output_berita.csv')
data_berita.head()

In [ ]:
import nltk
from nltk import sent_tokenize
nltk.download('punkt')

In [ ]:
sentence = sent_tokenize(data_berita['kalimat'][0])
print(sentence)
sentence_tokenize = pd.DataFrame(sentence)
sentence_tokenize.to_csv('sentence_berita.csv')

In [ ]:
import pandas as pd
data_berita = pd.read_csv('sentence_berita.csv')
data_kalimat = pd.DataFrame(data_berita)
data_berita.head()

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

count = CountVectorizer()
titles = pd.read_csv('sentence_berita.csv',sep=',',usecols=['kalimat'],squeeze=True)

docs = titles.values
bag = count.fit_transform(docs)
print(docs)

In [ ]:
cv_matrix = count.fit_transform(docs)
print(cv_matrix)

In [ ]:
print ("Term = ", (count.get_feature_names_out()))

**Matrix Similaritas**

In [ ]:
normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
print(normal_matrix.toarray())
print(normal_matrix.T.toarray)

**Membangun Graph**

In [ ]:
import networkx as nx

In [ ]:
res_graph = normal_matrix * normal_matrix.T
print(res_graph)

In [ ]:
nx_graph = nx.from_scipy_sparse_matrix(res_graph)

In [ ]:
nx.draw_circular(nx_graph)

**Menghitung Page Rank**

In [ ]:
ranks = nx.pagerank(nx_graph)

In [ ]:
for i in ranks:
    print(i, ranks[i])

In [ ]:
ranked_sentences = sorted(((ranks[i],s) for i,s in enumerate(docs)), reverse=True)

**Menampilkan 10 kalimat teratas**

In [ ]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])